# PFN-Pen

In [1]:
import torch
import numpy as np

import pfns4bo
from pfns4bo.scripts.acquisition_functions import TransformerBOMethod

# Select your test case
from test_functions.Ackley2D import Ackley2D, Ackley2D_Scaling
from PenaltyFunction import PenaltyFunction

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cpu:0")
dtype = torch.float32



## 1. Get the PFN model

In [2]:
# Load PFN
model_name = 'pfns4bo/final_models/model_hebo_morebudget_9_unused_features_3.pt'
PFN = TransformerBOMethod(torch.load(model_name), device=device)

## 2. PFN-Pen Bayesian Optimization 

In [3]:
# Set up constants
N_iter = 200 # Iteration number
rho = 1      # Initial rho for penalty function 
rho_scaling_factor = 1.5
Current_BEST = -1e10   # Some arbitrary very small number
Prev_BEST = -1e10
COUNT = 0


# Get initial random samples
trained_X = torch.rand(20,2)   # Initial samples (20 samples)

# Scale it to the domain of interest
X_Scaled = Ackley2D_Scaling(trained_X)

# Get the constraints and objective
trained_gx,trained_Y = Ackley2D(X_Scaled)

In [4]:

for ii in range(N_iter):

    # (0) Get the updated data for this iteration
    X_scaled = Ackley2D_Scaling(trained_X)
    trained_gx, trained_Y = Ackley2D(X_scaled)

    # (1) Randomly sample Xpen 
    X_pen = torch.rand(1000,trained_X.shape[1])

    # (2) Penalty Function
    PFfx = PenaltyFunction(trained_gx, trained_Y, rho)

    # (3) PFN inference phase with EI
    rec_idx, acq_ei = PFN.observe_and_suggest(X_obs=trained_X, y_obs=PFfx, X_pen=X_pen, return_actual_ei=True)

    # (4) Get the next search value
    best_candidate = X_pen[rec_idx,:].unsqueeze(0)

    # (5) Append the next search point
    trained_X = torch.cat([trained_X, best_candidate])

    Current_X = Ackley2D_Scaling(trained_X)
    Current_GX, Current_Y = Ackley2D(Current_X)


    # (6) Updating the rho for penalty function
    if ((Current_GX<=0).all(dim=1)).any():
        Current_BEST = torch.max(Current_Y[(Current_GX<=0).all(dim=1)])
    else:
        Current_BEST = Prev_BEST

    if Current_BEST <= Prev_BEST:
            COUNT += 1
    if COUNT >= 5:
        rho = rho*rho_scaling_factor
        COUNT = 0
        
    Prev_BEST = Current_BEST 


print( 'Best feasible value: {v}'.format(v=Current_BEST) )




Best feasible value: -2.655869483947754
